# <img src="https://github.com/JuliaLang/julia-logo-graphics/raw/master/images/julia-logo-color.png" height="100" /> _Colab Notebook Template_

## Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [1]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Plots Knet ArgParse"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=2
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2021-11-13 17:17:01 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Plots...
Installing Julia package Knet...
Installing Julia package ArgParse...
Installing Julia package CUDA...
Installing IJulia kernel...
[ Info: Installing julia kernelspec in /root/.local/share/jupyter/kernels/julia-1.6

Success! Please reload this page and jump to the next section.


# Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.30GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, haswell)
Environment:
  JULIA_NUM_THREADS = 2


In [3]:
using BenchmarkTools

M = rand(2048, 2048)
@benchmark M^2

BenchmarkTools.Trial: 10 samples with 1 evaluation.
 Range (min … max):  489.243 ms … 604.164 ms  ┊ GC (min … max): 0.22% … 19.00%
 Time  (median):     502.942 ms               ┊ GC (median):    0.11%
 Time  (mean ± σ):   514.367 ms ±  34.846 ms  ┊ GC (mean ± σ):  2.78% ±  5.96%

  ▁▁  ▁ █▁█                   ▁                               ▁  
  ██▁▁█▁███▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁█ ▁
  489 ms           Histogram: frequency by time          604 ms <

 Memory estimate: 32.00 MiB, allocs estimate: 2.

In [4]:
if ENV["COLAB_GPU"] == "1"
    using CUDA

    M_gpu = cu(M)
    @benchmark CUDA.@sync M_gpu^2
else
    println("No GPU found.")
end

BenchmarkTools.Trial: 692 samples with 1 evaluation.
 Range (min … max):  5.942 ms …  10.082 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     7.112 ms               ┊ GC (median):    0.00%
 Time  (mean ± σ):   7.216 ms ± 366.690 μs  ┊ GC (mean ± σ):  0.00% ± 0.00%

                               ▆█▆▃▁      ▅▅▃                  
  █▆▁▁▁▁▁▁▁▆▇▇▅▁▄▁▁▁▁▁▅▅▁▅▁▁▁▁███████▁█▅▇▇████▇▇▇█▆▆▁▄▆█▄▁▄▄▆ ▇
  5.94 ms      Histogram: log(frequency) by time      8.16 ms <

 Memory estimate: 208 bytes, allocs estimate: 8.

Named-entity recognition (NER) (also known as entity identification, entity chunking and entity extraction) is a subtask of information extraction that seeks to locate and classify named entities in text into pre-defined categories such as the names of persons, organizations, locations, expressions of times, quantities, monetary values, percentages, etc.

Most research on NER systems has been structured as taking an unannotated block of text, such as the following **example**:

**INPUT:** Jim bought 300 shares of Acme Corp. in 2006.

And producing an annotated block of text that highlights the names of entities:

**OUTPUT:** [Jim]Person bought 300 shares of [Acme Corp.]Organization in [2006]Time.

In this example, a person name consisting of one token, a two-token company name and a temporal expression have been detected and classified.(Wikipedia)

Your task in this lab is to implement named entity LSTM based tagger which uses an LSTM to extract features from the input sentence, which are then passed through a multi-layer perceptron to predict
the tag of the word. Finally, train that model on [WikiNER](https://github.com/neulab/dynet-benchmark/tree/master/data/tags) dataset.

In [9]:
import Pkg; Pkg.add("IterTools")

    Updating registry at `~/.julia/registries/General`
   Resolving package versions...
    Updating `~/.julia/environments/v1.6/Project.toml`
  [c8e1da08] + IterTools v1.3.0
  No Changes to `~/.julia/environments/v1.6/Manifest.toml`


In [10]:
using Printf, Dates, Random, CUDA, Knet, ArgParse, Test, Base.Iterators, IterTools

STDOUT = Base.stdout

import Knet: train!
include(joinpath(Knet.dir(), "data", "wikiner.jl"))
_atype = CUDA.functional() ? KnetArray{Float32} : Array{Float32}

@info "Adding required packages and importing WikiNER dataset"

┌ Info: Adding required packages and importing WikiNER dataset
└ @ Main In[10]:9


## Prepare samples for the network
Your first task is to prepare instances for the network. We're given with the tokens (words and tags) and we need to make them understandable by our neural network. For this purpose, we build vocabularies (for both words and tags) and construct vocabulary to index dictionaries by using those vocabularies (w2i and t2i, word2index, tag2index). Then, we convert words and tags to indices with the usage of our dictionaries.

```julia
julia> show_instance() # show instance in not implemented in Knet, it is a hypothetical procedure
Inputs sentence:
Sent-> That inscribed in the genealogical records of his family is Jiang Zhoutai .
NERs-> O    O         O  O   O            O       O  O   O      O  I-PER I-PER   O

Timesteps:
Time step 1 ---> Inputs: That
                 Outputs: O
Time step 2 ---> Inputs: inscribed
                 Outputs:O
Time step 3 ---> Inputs: in
                 Outputs: O
Time step 4 ---> Inputs: the
                 Outputs: O
Time step 5 ---> Inputs: genealogical
                 Outputs: O
Time step 6 ---> Inputs: records  .
                 Outputs: O
Time step 7 ---> Inputs: of
                 Outputs: O
Time step 8 ---> Inputs: his
                 Outputs: O
Time step 9 ---> Inputs: family
                 Outputs: O
Time step 10 --->Inputs: is
                 Outputs: O
Time step 11 ---> Inputs: Jiang
                  Outputs: I-PER
Time step 12 ---> Inputs: Zhoutai
                  Outputs: I-PER
Time step 13 ---> Inputs: .
                  Outputs: O
```

Our input and output arrays should be integers instead of texts.

In this step, you need to implement `make_instance` function
instance is a list of tuples. Each tuple contains a word and the corresponding tag as string.
You need to convert them into indices using word to index (w2i) and tag to index (t2i)

In [49]:
"""
    make_instance(instance, w2i, t2i)

Return tuple of two sequences containing inputs and the corresponding outputs respectively.

This function does this by converting each input unit in the instance into its corresponding value in w2i, and does the same for output units using t2i.
"""
function make_instance(instance, w2i, t2i, unk=UNK)
    input = Array{Int}([])
    output = Array{Int}([])
    # Your code here
    for l = 1:length(instance)
        push!(input, get!(w2i, instance[l][1], 17516))
        push!(output, get!(t2i, instance[l][2], 6))
    end
    return input, output
end


"""
   make_instances(data, w2i, t2i)

Iterate over `data` and Return `words` and `tags`
"""
function make_instances(data, w2i, t2i)
    words = []; tags = []
    for k = 1:length(data)
        this_words, this_tags = make_instance(data[k], w2i, t2i)
        push!(words, this_words)
        push!(tags, this_tags)
    end
    return words, tags
end

@info "Testing instances"
data = WikiNERData();
dev = make_instances(data.dev, data.w2i, data.t2i);
@test dev[1][2][3] == 22450
@test size.(dev) == ((1696,), (1696,))

┌ Info: Testing instances
└ @ Main In[49]:35


Test Passed

### WikiNERProcessed
This struct contains processed data (e.g words and tags are indices)
and necessary variables to prepare minibatches.
WikiNERProcessed struct works as a data iterator, which will you implement in the next step.

In [50]:
mutable struct WikiNERProcessed
    words
    tags
    batchsize
    ninstances
    shuffled
end

"""
   WikiNERProcessed(instances, w2i, t2i; batchsize=16, shuffled=true)

Return a WikiNERProcessed object with the given instances
"""
function WikiNERProcessed(instances, w2i, t2i; batchsize=16, shuffled=true)
    words, tags = make_instances(instances, w2i, t2i)
    ninstances = length(words)
    return WikiNERProcessed(words, tags, batchsize, ninstances, shuffled)
end

@info "WikiNERProcessed"
devdata = WikiNERProcessed(data.dev, data.w2i, data.t2i; shuffled=false);
@test devdata.words[1][1] == 17516
@test length(devdata.words) == 1696

┌ Info: WikiNERProcessed
└ @ Main In[50]:20


Test Passed

### WikiNERProcessed Iterator
Please note that this function returns tuple of two tuples.

The first one contains a data batch with words as an input for our model, and tags as the corresponding output, and batchsizes of this batch.
Since you will use the RNN callable object in your model.
It supports variable length instances in its input.
However, you need to prepare your input such as the RNN object can work on it. See `batchSizes` option of the RNN object using `@doc RNN` and Look up `zeros`, `sortperm`, `min`

In [106]:
"""
    iterate(d::WikiNERProcessed[, state])

Iterate over `d::WikiNERProcessed` object. If `state` is missing, it's the beginning
of the whole iteration process. 
WikiNERProcessed(words, tags, batchsize, ninstances, shuffled)

for each batch:
    define input_array
    sort sequences by their length (longer first)
    for each timestep_i in longest sample
        for each sequence s in ordered_sequences
             add unit_i of sequence s to our input array

"""
function Base.iterate(d::WikiNERProcessed, state=ifelse(d.shuffled, randperm(d.ninstances), 1:d.ninstances))
    # Your code here
    this_words= d.words[state]
    this_tags= d.tags[state]
    words = Array{Int}([]); tags = Array{Int}([])
    if size(state,1)<d.batchsize
        these_words = this_words[1:end]
        these_tags =  this_tags[1:end]
        
        these_lengths = length.(these_words)
        these_lengths_sorted = sortperm(these_lengths,rev=true)
        
        those_words = these_words[sortperm(these_lengths,rev=true)]
        those_tags = these_tags[sortperm(these_lengths,rev=true)]
        those_lengths = length.(those_words)
        
        for m in 1:those_lengths[1]
            for n in 1:size(state,1)
                try
                    push!(words, those_words[n][m])
                    push!(tags, those_tags[n][m])
                catch
                end
            end
        end
    else
        these_words = this_words[1:d.batchsize]
        these_tags =  this_tags[1:d.batchsize]
        these_lengths = length.(these_words)
        these_lengths_sorted = sortperm(these_lengths,rev=true)

        
        those_words = these_words[these_lengths_sorted]
        those_tags = these_tags[these_lengths_sorted]
        those_lengths = length.(those_words)

        batchsizes = Array{Int}(zeros(those_lengths[1]))
        for m in 1:those_lengths[1]
            for n in 1:d.batchsize
                try
                    push!(words, those_words[n][m])
                    batchsizes[m] = batchsizes[m] + 1 
                    push!(tags, those_tags[n][m])
                catch
                end     
            end
        end 
    end
    state = state[d.batchsize+1:end]
    if state == []
        return nothing
    end
    
    return ((words, tags, batchsizes), state)
end

Base.IteratorSize(::Type{WikiNERProcessed}) = Base.SizeUnknown()
Base.IteratorEltype(::Type{WikiNERProcessed}) = Base.HasEltype()

@info "Testing WikiNERProcessed Iterator"
((words, tags, batchsizes), new_state) = iterate(devdata);

@test length.((words, tags, batchsizes)) == (397, 397, 55)
@test new_state == 17:1696

┌ Info: Testing WikiNERProcessed Iterator
└ @ Main In[106]:75


Test Passed

## Model Components implementation

### Embedding layer
This layer maps each vocabulary to its corresponding vector using its Int id. It works with mini-batches.

In [95]:
"""
    Embedding(vocabsize::Int, embedsize::Int, atype=_atype, scale=0.01)

Create a Embedding layer and initialize its weight. Initial weight parameters are
sampled from normal distribution scaled by a `scale` factor.

# Examples
```julia-repl
julia> embed = Embedding(100, 25);

julia> x = rand(1:10, 10);

julia> embed(x); # forward call
```
"""
mutable struct Embedding
    w # weight
end

function Embedding(vocabsize::Int, embedsize::Int, atype=_atype, scale=0.01)
    w = Param(convert(atype, scale*randn(embedsize, vocabsize)));
    return Embedding(w)
end


function (l::Embedding)(x)
    l.w[:, x]
end



@info "Testing embedding layer"
Random.seed!(1)
embed = Embedding(100, 25);
x = rand(1:25, 12, 32);
@test size(embed(x)) == (25, 12, 32)
@test sum(embed(x)) ≈ -5.08327f0

┌ Info: Testing embedding layer
└ @ Main In[95]:32


Test Passed

### Linear layer

In [79]:
"""
    Linear(inputsize, outputsize; atype=Array{Float64}, scale::Float64=0.1)

Create a linear layer with its weight and bias. Initial weight parameters are
sampled from normal distribution scaled by a `scale` factor. Initial bias
values are zeros.

# Examples
```julia-repl
julia> layer = Linear(50, 10);

julia> x = rand(2, 50);

julia> layer(x); # forward call
```

struct Linear; w; b; end

Linear(input::Int, output::Int)=Linear(param(output,input), param0(output))

(l::Linear)(x) = l.w * mat(x,dims=1) .+ l.b  # (H,B,T)->(H,B*T)->(V,B*T)


"""
mutable struct Linear
    w # weight
    b # bias

    function Linear(inputsize, outputsize; atype=_atype, scale::Float64=0.01)
        # Your code here
        w = Param(convert(atype,scale*randn(outputsize,inputsize)))
        #w = Param(scale*randn(outputsize,inputsize))
        b = Param(convert(atype,zeros(outputsize,1)))
        new(w,b)
    end
end

function (l::Linear)(x)
    # Your code here
    l.w * mat(x,dims=1) .+ l.b  # (H,B,T)->(H,B*T)->(V,B*T)
end



@info "Testing linear layer"
Random.seed!(1)
lin = Linear(100, 200);
x = _atype(randn(100, 32));
@test size(lin(x)) == (200, 32)
@test sum(lin(x)) ≈ -3.8317218f0

┌ Info: Testing linear layer
└ @ Main In[79]:45


Test Passed

### Hidden layer

In [80]:
"""
    Hidden(inputsize, outputsize, fun=relu, atype=_atype, scale=0.1)

Create a hidden layer with its weight and bias and activation function. Initial weight parameters are
sampled from normal distribution scaled by a `scale` factor. Initial bias
values are zeros.

# Examples
```julia-repl
julia> layer = Hidden(100, 200);

julia> x = rand(100, 5);

julia> layer(x); # forward call
```
"""
mutable struct Hidden
    w # weight
    b # bias
    fun # non-linear activation function like relu or tanh

    function Hidden(inputsize, outputsize, fun=relu, atype=_atype, scale=0.1)
        # Your code here
        w = Param(convert(atype,scale*randn(outputsize,inputsize)))
        b = Param(convert(atype,zeros(outputsize,1)))
        fun = relu
        new(w,b,relu)
    end
end

function (l::Hidden)(x)
    # Your code here
    max.(0,(l.w * mat(x,dims=1) .+ l.b))  # (H,B,T)->(H,B*T)->(V,B*T)
    #relu(l.w * mat(x,dims=1) .+ l.b)
end

@info "Testing hidden layer"
Random.seed!(1)
hid = Hidden(200, 256);
x = _atype(randn(200, 32));
println(size(hid(x)))
@test size(hid(x)) == (256, 32)
@test sum(hid(x)) ≈ 4635.545f0

┌ Info: Testing hidden layer
└ @ Main In[80]:37


(256, 32)


Test Passed

### NER Tagger model

Our model consists of four layers. Size of their outputs are as the following:
* **(T)** - Input
* **(E, T)** - Embedding
* **(RNN, T)** - RNN
* **(H, T)** - Hidden
* **(NTags, T)** - Projection

    #m.embed(m.rnn(m.hidden(m.projection(x)));batchsizes)
    a = m.embed(x)
    @show size(x,1)
    @show size(a)
    b = m.rnn(a)
    @show size(b)
    c = m.hidden(b)
    @show size(c)
    d = m.projection(c)
    @show size(d)

In [101]:

mutable struct NERTagger
    embed::Embedding
    rnn::RNN
    hidden::Hidden
    projection::Linear
end

function NERTagger(no_words, no_tags, embed_size, rnn_hidden_size, mlp_hidden_size, atype=_atype)
    # Your code here
    embed = Embedding(no_words, embed_size)
    rnn = RNN(embed_size, rnn_hidden_size,atype=_atype)
    hidden = Hidden(rnn_hidden_size, mlp_hidden_size)
    projection = Linear(mlp_hidden_size, no_tags,atype=_atype)

    return NERTagger(embed, rnn, hidden, projection)
end

function (m::NERTagger)(x; batchsizes=nothing)
    # Your code here
    m.projection(m.hidden(m.rnn(m.embed(x);batchSizes=batchsizes)))
end

@info "Testing forward pass of NERTagger"
Random.seed!(1)
nwords, ntags = length(data.w2i), data.ntags
model = NERTagger(nwords, ntags, 128, 50, 32)
output = model(words; batchsizes=batchsizes)

@test size(output) == (9, 397)
@test sum(output) == 0.4713313f0

┌ Info: Testing forward pass of NERTagger
└ @ Main In[101]:24


Test Failed at In[101]:31
  Expression: sum(output) == 0.4713313f0
   Evaluated: 0.25421953f0 == 0.4713313f0


LoadError: ignored

Now you will implement loss function for your model.
Firstly get your probabilities from your model.
Then calculate the loss function for average per token. You can use `nll` for this purpose.

In [102]:
function (m::NERTagger)(x, ygold, batchsizes; average=true)
    # Your code here
    nll(m(x),ygold)
end

@info "Testing loss function of NERTagger"
Random.seed!(1)
nwords, ntags = length(data.w2i), data.ntags
model = NERTagger(nwords, ntags, 128, 50, 32)
l = model(words, tags, batchsizes)

@test l ≈ 2.198382f0

┌ Info: Testing loss function of NERTagger
└ @ Main In[102]:6


Test Failed at In[102]:12
  Expression: l ≈ 2.198382f0
   Evaluated: 2.1972032f0 ≈ 2.198382f0


LoadError: ignored

### Loss for a whole dataset

Define a `loss(model, data)` which returns a `(Σloss, Nloss)` pair if `average=false` and
a `Σloss/Nloss` average if `average=true` for a whole dataset. Assume that `data` is an
iterator of `(words, gold_tags, batchsizes)` such as `WikiNERProcessed` and `model(x,y;average)` is a model like
`NERTagger` that computes loss on a single `(x,y)` pair.

In [114]:
"""
    loss(model::NERTagger, data; average=true)

Return overall loss of model on data.
"""
function loss(model::NERTagger, data; average=true)
    l = 0
    n = 0
    for (x, y, b) in data
        # Your code here

        l = l + model(x, y, b)
        n = n + 1
    end
    average && return l / n
    return l, n
end

@info "Testing loss function"
Random.seed!(1)
@test loss(model, devdata) ≈ 2.1978726f0
@test loss(model, devdata; average=false) == (85735.13f0, 39007)

┌ Info: Testing loss function
└ @ Main In[114]:19


Test Failed at In[114]:22
  Expression: loss(model, devdata; average = false) == (85735.13f0, 39007)
   Evaluated: (230.70518f0, 105) == (85735.13f0, 39007)


LoadError: ignored

### Question
Why are we getting such value for loss? is this expected and why?

Write your answer here:

-

### Accuracy metric
This function will be the metric which will evaluate our model's performance.

You will iterate over the given `data` object, predicting each instance and adding number of correctly predicted tokens to `ncorrect` and the number of tokens to `ntokens`.

possible helpful procedures: `argmax`, `vec`

In [137]:
@doc reshape

```
reshape(A, dims...) -> AbstractArray
reshape(A, dims) -> AbstractArray
```

Return an array with the same data as `A`, but with different dimension sizes or number of dimensions. The two arrays share the same underlying data, so that the result is mutable if and only if `A` is mutable, and setting elements of one alters the values of the other.

The new dimensions may be specified either as a list of arguments or as a shape tuple. At most one dimension may be specified with a `:`, in which case its length is computed such that its product with all the specified dimensions is equal to the length of the original array `A`. The total number of elements must not change.

# Examples

```jldoctest
julia> A = Vector(1:16)
16-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
 14
 15
 16

julia> reshape(A, (4, 4))
4×4 Matrix{Int64}:
 1  5   9  13
 2  6  10  14
 3  7  11  15
 4  8  12  16

julia> reshape(A, 2, :)
2×8 Matrix{Int64}:
 1  3  5  7   9  11  13  15
 2  4  6  8  10  12  14  16

julia> reshape(1:6, 2, 3)
2×3 reshape(::UnitRange{Int64}, 2, 3) with eltype Int64:
 1  3  5
 2  4  6
```


In [181]:
@doc repeat

```
repeat(A::AbstractArray, counts::Integer...)
```

Construct an array by repeating array `A` a given number of times in each dimension, specified by `counts`.

# Examples

```jldoctest
julia> repeat([1, 2, 3], 2)
6-element Vector{Int64}:
 1
 2
 3
 1
 2
 3

julia> repeat([1, 2, 3], 2, 3)
6×3 Matrix{Int64}:
 1  1  1
 2  2  2
 3  3  3
 1  1  1
 2  2  2
 3  3  3
```

```
repeat(A::AbstractArray; inner=ntuple(x->1, ndims(A)), outer=ntuple(x->1, ndims(A)))
```

Construct an array by repeating the entries of `A`. The i-th element of `inner` specifies the number of times that the individual entries of the i-th dimension of `A` should be repeated. The i-th element of `outer` specifies the number of times that a slice along the i-th dimension of `A` should be repeated. If `inner` or `outer` are omitted, no repetition is performed.

# Examples

```jldoctest
julia> repeat(1:2, inner=2)
4-element Vector{Int64}:
 1
 1
 2
 2

julia> repeat(1:2, outer=2)
4-element Vector{Int64}:
 1
 2
 1
 2

julia> repeat([1 2; 3 4], inner=(2, 1), outer=(1, 3))
4×6 Matrix{Int64}:
 1  2  1  2  1  2
 1  2  1  2  1  2
 3  4  3  4  3  4
 3  4  3  4  3  4
```

```
repeat(s::AbstractString, r::Integer)
```

Repeat a string `r` times. This can be written as `s^r`.

See also: [`^`](@ref :^(::Union{AbstractString, AbstractChar}, ::Integer))

# Examples

```jldoctest
julia> repeat("ha", 3)
"hahaha"
```

```
repeat(c::AbstractChar, r::Integer) -> String
```

Repeat a character `r` times. This can equivalently be accomplished by calling [`c^r`](@ref :^(::Union{AbstractString, AbstractChar}, ::Integer)).

# Examples

```jldoctest
julia> repeat('A', 3)
"AAA"
```


In [191]:
"""
    accuracy(model::NERTagger, data, i2t)
    push!(output, get!(t2i, instance[l][2], 6))
Return accuracy of tags given a model and dataset
"""
function accuracy(model::NERTagger, data, i2t)
    ncorrect = 0
    ntokens = 0

    for (x, ygold, batchsizes) in data
        scores = model(x; batchsizes=batchsizes)
        # Your code here
        a = vec(argmax(scores, dims=1))
        bb = repeat( (1:size(scores,1)) , 1, size(scores,2))
        ncorrect = ncorrect + sum(bb[a] .== ygold)
        ntokens = ntokens + length(ygold)
        
        
        if ntokens == 1
        return ncorrect / ntokens
        end

    end

    return ncorrect / ntokens
end

@info "Testing accuracy function"
accuracy(model, devdata, data.i2t)
@test accuracy(model, devdata, data.i2t) ≈ 0.0239700566564975

┌ Info: Testing accuracy function
└ @ Main In[191]:28


Test Failed at In[191]:30
  Expression: accuracy(model, devdata, data.i2t) ≈ 0.0239700566564975
   Evaluated: 0.22206155752943607 ≈ 0.0239700566564975


LoadError: ignored

The following function can be used to train our model. trn is the training data, dev is used to determine the best model, tst... can be zero or more small test datasets for loss reporting. It returns the model that does best on dev.

In [192]:
"""
    train!(model, trn, dev, tst...)

Train `model` on `trn` data with Adam optimizer and Return the best performing model on `dev` data.
"""
function train!(model, trn, dev, tst...)
    bestmodel, bestloss = deepcopy(model), loss(model, dev)
    progress!(adam(model, trn), steps=1000) do y
        losses = [ loss(model, d) for d in (dev,tst...) ]
        if losses[1] < bestloss
            bestmodel, bestloss = deepcopy(model), losses[1]
        end
        return (losses...,)
    end
    return bestmodel
end

Knet.Train20.train!

## Training the model
Here we train our model for 10 epochs using the previous procedure. You can try to fiddle with the hyperparameters i.e. (embed_size, hidden_size, epochs, etc..) to get better loss on dev set. You should get a value of dev loss around `0.26`.

In [193]:
@info "Training NERTagger model"
@info "Seeding random number generator"
Random.seed!(1)

@info "Loading data"
data = WikiNERData();
dtrn = WikiNERProcessed(data.trn, data.w2i, data.t2i);
ddev = WikiNERProcessed(data.dev, data.w2i, data.t2i; shuffled=false);
epochs = 1; @show epochs
ctrn = [ b for b in dtrn ]
trnx10 = collect(flatten(shuffle!(ctrn) for i in 1:epochs))
trn20 = ctrn[1:20]
dev = [ b for b in ddev ]

@info "Initializing model"

@show nwords
@show ntags
embed_size = 128; @show embed_size
rnn_size = 50; @show rnn_size
hidden_size = 32; @show hidden_size
model = NERTagger(nwords, ntags, embed_size, rnn_size, hidden_size)

# train the model (one epoch should take around 2 mins on gpu):
model = train!(model, trnx10, dev, trn20)

┌ Info: Training NERTagger model
└ @ Main In[193]:1
┌ Info: Seeding random number generator
└ @ Main In[193]:2
┌ Info: Loading data
└ @ Main In[193]:5


epochs = 1
nwords = 30876
ntags = 9
embed_size = 128
rnn_size = 50
hidden_size = 32


┌ Info: Initializing model
└ @ Main In[193]:15
┣████████████████████┫ [100.00%, 8884/8884, 02:49/02:49, 52.63i/s] (0.3259385f0, 0.18131289f0)


NERTagger(Embedding(P(Knet.KnetArrays.KnetMatrix{Float32}(128,30876))), LSTM(input=128,hidden=50), Hidden(P(Knet.KnetArrays.KnetMatrix{Float32}(32,50)), P(Knet.KnetArrays.KnetMatrix{Float32}(32,1)), Knet.Ops20.relu), Linear(P(Knet.KnetArrays.KnetMatrix{Float32}(9,32)), P(Knet.KnetArrays.KnetMatrix{Float32}(9,1))))

## Evaluation of the best model
**Expected Values**
- Development loss = 0.25991333
- Development accuracy = 0.9176301689440357
- Training loss = 0.11450425
- Training accuracy = 0.9643299845754065

In [194]:
@info "Evaluating the model"

dloss = loss(model, ddev)
tloss = loss(model, dtrn)
dacc = accuracy(model, ddev, data.i2t)
tacc = accuracy(model, dtrn, data.i2t)

println("Development loss = ", dloss)
println("Development accuracy = ", dacc)
println("Training loss = ", tloss)
println("Training accuracy = ", tacc)

┌ Info: Evaluating the model
└ @ Main In[194]:1


Development loss = 0.3259385
Development accuracy = 0.843911381945879
Training loss = 0.18757011
Training accuracy = 0.8719183429463659


Add new code cells by clicking the `+ Code` button (or _Insert_ > _Code cell_).

Have fun!

<img src="https://raw.githubusercontent.com/JuliaLang/julia-logo-graphics/master/images/julia-logo-mask.png" height="100" />